In [54]:
%load_ext autoreload
%autoreload 2

from train import *
from models import *
from tools import *

from keras import optimizers
from keras.models import load_model
from keras.utils import to_categorical
from keras.callbacks import *
from keras.preprocessing.image import ImageDataGenerator

from PIL import Image
from IPython.display import display

import numpy as np
import os
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
X_train, y_train = load_data()
X_val, y_val = load_data("val")

input_shape = X_train[0].shape
print("Input shape: " + str(input_shape))
print(X_train[0][0][0])

y_train = to_categorical(y_train, 200)
y_val = to_categorical(y_val, 200)

Loading train data

Finished loading train data

Loading val data

Finished loading val data

Input shape: (64, 64, 3)
[0.75294118 0.83137255 0.83529412]


In [30]:
labels = get_label_dict()
words = get_word_labels()

Loading labels

Done

Loading words

Done



In [56]:
# Image preprocessing for the data
datagen = ImageDataGenerator(horizontal_flip=True, 
                             shear_range=20., 
                             zoom_range=0.4, 
                             width_shift_range=0.2,
                             height_shift_range=0.2, 
                             channel_shift_range=0.1, 
                             fill_mode="reflect", 
                             preprocessing_function=shuffle_channels)

testgen = ImageDataGenerator()

data = {"train": (X_train, y_train), 
        "val": (X_val, y_val)}

In [66]:
def lr_tune(model, optim_class):
    results = []
    
    for _ in range(5):
        keras.backend.clear_session()

        lr = random.uniform(1e-4, 1e-1)
        optim = optim_class(learning_rate=lr)

        history = train(model_path=model_path, 
                        restore=False, 
                        epochs=10,
                        model=model, 
                        optim=optim,
                        datagen=datagen, 
                        testgen=testgen, 
                        data=data, 
                        cb_list=None, 
                        batch_size=32, 
                        verbosity=0)

        history = history.history

        train_acc = history["accuracy"][-1]
        val_acc = history["val_accuracy"][-1]

        result = "lr: {}, train: {}, val: {}".format(lr, train_acc, val_acc)
        results.append(result)
        print(result)
        
    return results

In [ ]:
optim_list = [optimizers.Adam, optimizers.RMSProp, optimizers.Adadelta, optimizers.Ftrl]

for optim in optim_list:
    print(optim.name)
    results = lr_tune(BravoNet(input_shape, "v3", quiet=True), optim)
    file = open("tuning/" + optim.name + ".txt", "w")
    file.write(results)
    file.close()